# Artificial Intelligence 1 Week 4 Practical
### Goals
This session is split into two parts.

The first activity is designed  to give you practical experience of:
- setting bot predicates outside your bot,  to customize it
- using `<system>` tag-pairs to make calls and connecting your chatbot to external services.

The second activity gives you a chance to work on the first bit of coursework, and make sure your testing toolchain is in place.


### By the end of this session you should have

- Understood how external programmes can affect your bot,  and vice versa


- Understood what you need to do for the first piece of coursework.
- Checked your toolchain for developing the coursework.

### Before next week you should have
- Expanded your revision chatbot so it can the same question asked in different ways about each of the facts you have stored.   
- Expanded the domain-specific knowledge contents of your revision bot with definitions and examples for all the other key concepts covered so far.

### Additional Resources
- Pandorabots (www.pandorabots.com) offers free sign-in and hosts a web-based interface for authoring bots.   
   Once you start doing more complex things, the feedback on what is happening as your queries are processed  is a little better than the python aiml version.  
   But some students have found the text preprocessing is  different to the 'official reference' version we use for the marking system.
- Program-y is a fully featured AIML2-compliant python-based system with clients for twitter, etc. 
   You might like to try developing your AIML in pandorabots then importing it into these jupyter notebooks.
   
   
- Good places to look for help with AIML syntax are: 
 - [Pandorabots AIML-Fundaments](https://www.pandorabots.com/docs/aiml-fundamentals)
 - [Pandorabots AIML reference - for syntax](https://www.pandorabots.com/docs/aiml-reference/)
 - [The official AIML site](http://www.aiml.foundation/doc.html)

<div class="alert alert-block alert-warning"> <b>REMEMBER:</b> Because it is much more lightweight, in class we are using the python-aiml package. <br>This is only AIMLv1-compliant, and  does not support all the features of AIMLv2. </div>

<div class="alert alert-block alert-danger"> <b>REMEMBER:</b> IF you are running the notebooks on the cloud server you need to click on the kernel menu and then change-kernel to'AIenv'.<br>
IF you are running locally AND you created a virtual environment in Jupyter click on the kernel menu then change-kernel OR in VSCode use the kernel selector in the top-right hand corner.
</div>

# Tutorial activity 1: Communications between your bot and other programmes

AIML V2 comes with new tags such as `<sraix>` that send messages to other programmes either on the local machine, or running as a web service.

However even in AIML v1, there is the option to make a "system" call, which works in the same way as running a command from a terminal (mac/linux) or the "command tool" (windows)

## Activity 1.1 Setting/Getting Bot variables from outside the bot, and using `<system>` tags



### Start by reading then running the 3 cells below which:
1. Create a bot 
2. Demonstrate how an external programme can tell a running bot to set some predicate values according to the computer it is hosted on.   
In this case some of the stored values are the names of programmes to run outside the bot
3. Demonstrate how an external programme can  retrieve short term memory (predicates) from a running bot 

In [ ]:
# Run this cell to create  your bot
import aiml
from IPython.display import HTML
import platform

# Create the kernel and learn AIML files
myChatbot = aiml.Kernel()


In [ ]:
## In this cell python code is telling a running bot to set some values using the setPredicate() method

# use a standard python library to find out what operating system this code is running on
osname = platform.system()


# set some command  according to the operating system
if (osname=='Darwin'): #Macos
    openUrlCommand = "open"
    getDateCommand = "date | cut -d' ' -f1-3,6"
    getTimeCommand = "date|cut -d' ' -f4,5"
elif(osname=='Windows'):
    openUrlCommand = "start"
    getDateCommand = "date/T"
    getTimeCommand = "time/T"
else:
    print("There may not be a command-line command to open an url on your system.\n On linux it depends what you have installed")
    openUrlCommand = ""
    getDateCommand= "date | cut -d' ' -f1-3,6"
    getTimeCommand = "date|cut -d' ' -f4,5"

    
# "teach" the bot these values by creating  predicates
myChatbot.setPredicate("osname", osname)
myChatbot.setPredicate("getDateCommand", getDateCommand)
myChatbot.setPredicate("getTimeCommand", getTimeCommand)
myChatbot.setPredicate("openURLCommand", openUrlCommand)

In [ ]:
## In this cell python code asks the bot for values of variables (predicates) using the getPredicate() method


print("The bot has its variable osname set to " + myChatbot.getPredicate("osname"))

print( "The bot will get the date calling system command: " + myChatbot.getPredicate("getDateCommand")  )
print( "The bot will get the time by calling system command: " + myChatbot.getPredicate("getTimeCommand") )

print("The bot will use this command to open a url: " + myChatbot.getPredicate("openURLCommand"))

### Next read, understand, then run the  cell below to write some categories to file
The categories contain the knowlege about  how to answer  these things in an interactive chat session

The first one just retrieves a stored value and returns it.

The second and third categories  use `<system>` calls to the host computer, retrieving  stored values to set the commands to be run

In [ ]:
%%writefile "simple_extensions.aiml"
<aiml version="1.0.1" encoding="UTF-8">

<category>
    <pattern> WHAT OPERATING SYSTEM ARE YOU RUNNING ON</pattern>
    <template><get name="osname" /></template>
</category>

<category>
    <pattern> WHAT IS THE TIME </pattern>
    <template> 
          It is <system><get name = "getTimeCommand"/></system>
    </template>
</category>

<category>
    <pattern> WHAT IS THE DATE </pattern>
    <template> 
        Today is <system><get name="getDateCommand"/></system>
    </template>
</category>



</aiml>

### Now its time to experiment:  run a chat session and ask the bot 
- what time it is
- what date it is
-  what operating sysem it is running on.

In [ ]:
myChatbot.learn("simple_extensions.aiml")
myChatbot.verbose(True)

keepGoing=True
while(keepGoing):
    nextInput = input("Enter your message >> ")
    if(nextInput=='bye'):
        keepGoing=False
    else:
        print (myChatbot.respond(nextInput))





## Activity 1.2 Connecting the bot to query other web services


### First read, then run the cell below to store some categories which make calls to external services
Each one does the same three things in its template:

1. Uses a `<think>` pair to construct a variable called `runquery` by adding together
  - the computer-specific command to open a url in a new tab
  - the name of a web service
  - the thing the user asked for - stored in `<star/>`
2. Prints out a message and the hyperlink it hs constructed.
3. Uses a `<system>` tag-pair to ask the host computer to perform that action 

In [ ]:
%%writefile "web_extensions.aiml"
<aiml version="1.0.1" encoding="UTF-8">



<category>
    <pattern> GOOGLE *</pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/search?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
  <pattern> IMAGES *</pattern>
  <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/images?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> MAP OF *</pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/maps?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> WIKIPEDIA * </pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> https://en.wikipedia.org/wiki/<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> CALCULATE * </pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/search?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

</aiml>

### Now try these out by running the next cell 
**On your own machine** these should:
- open a new tab, and 
- produce a hyperlink in the chat window.

**If you are using the csctcloud server**  then 
- you will still get the hyperlink in the chat window, which should open a new tab in your browser when you click on it. 
- but for security reasons it's harder to (or Jim doesn't know how to) make a remote machine (csctcloud) open a tab in your browser
</div>

In [ ]:
myChatbot.learn("web_extensions.aiml")
myChatbot.verbose(True)

keepGoing=True
while(keepGoing):
    nextInput = input("Enter your message >> ")
    if(nextInput=='bye'):
        keepGoing=False
    else:
        print (myChatbot.respond(nextInput))




<div class = "alert alert-block alert-info"><b>Congratulations:</b>  You have now taken the first steps to personalising a bot.<br>
Below are some suggestions for things that you could do in your self-study activities.</div>

## Activity 1.3 Suggestions for further work

Below are some suggestions for things that you could do in the remaining tutorial time or in your self-study activities.

You should aim to complete at least one of them.

1. Extend the categories above to link to other sources beyond google and wikipedia.
2. Refactor the AIML categories above, to make a logical distinction between 
 - the category that formulates the query, and 
 - the category that runs the query on the host system.  
 - You could do this using `<think>` to make and save a query for different services,  
   and then use `<srai>` to call  a second category that ran the query and deals with the operating system.
 This should allow you to change things more flexibly in the future. 
  
3. Use the function myChatBot.setPredicate() to give your bot a name and personality, and write categories that answer user questions about the bot.
4. Make 2 bot instances with different names and see if you can get them talking to each other!



# Tutorial Activity 2:  20 minutes: Setting up your toolchain for the coursework.

**If you set up the coursework last week skip to the next cell**

1. If you have not already done so, create a new folder to store your first coursework code in.
2. Go to the assessments folder on Blackboard
and download these files into your coursework folder:
 - 1cat.aiml
 - AIML_Coursework_marker.ipynb
 - coursework-questions-and-responses.txt
   
3. Make a copy of the file 1cat.aiml with the name "student.aiml".  
 - by default the marking code will load your coursework from "student.aiml".  
   However, you will find instructions on how to change this within the notebook.
4. Run the notebook to test your toolchain, noting that: 
 - The marking system will tell you how many categories have been loaded, so you can check if your aiml file loaded as expected.
 - The file 1cat.aiml only has one category in and should score 1 mark.
 - There is a debug flag set to False.   Changing this to True will give you more feedback.
 - The marks and feedback you will get from the 'official' system are enclosed in xml tag-pairs ( &lt;SCORE&gt; and &lt;MESSAGE&gt; respectively).
 - The system creates a file student-results.txt which tells you question-by-question what the expected response was,  and what your bot responded.
 - **The marking system will randomise the question order every time** ( except the 3 questions that require context)
 

<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>